In [8]:
import os
# Find the latest version of spark 3.x  from http://www.apache.org/dist/spark/ and enter as the spark version
# For example:
# spark_version = 'spark-3.5.0'
spark_version = 'spark-3.5.0'
os.environ['SPARK_VERSION']=spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-11-jdk-headless -qq > /dev/null
!wget  http://www.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop3.tgz

print('---- unzip it -----')
!tar xf $SPARK_VERSION-bin-hadoop3.tgz
!pip install -q findspark

# Set Environment Variables
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop3"

# Start a SparkSession
#import findspark
#findspark.init()

Hit:1 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease
Hit:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease
Hit:3 http://security.ubuntu.com/ubuntu jammy-security InRelease
Hit:4 http://archive.ubuntu.com/ubuntu jammy InRelease
Hit:5 http://archive.ubuntu.com/ubuntu jammy-updates InRelease
Hit:6 http://archive.ubuntu.com/ubuntu jammy-backports InRelease
Hit:7 https://ppa.launchpadcontent.net/c2d4u.team/c2d4u4.0+/ubuntu jammy InRelease
Hit:8 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Hit:9 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Hit:10 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Reading package lists... Done
--2023-11-21 21:10:25--  http://www.apache.org/dist/spark/spark-3.5.0/spark-3.5.0-bin-hadoop3.tgz
Resolving www.apache.org (www.apache.org)... 151.101.2.132, 2a04:4e42::644
Connecting to www.apache.org (www.apache.org)|151.101.2.132|:8

In [9]:
# Start a SparkSession
import findspark
findspark.init()

In [10]:
# Import packages
from pyspark.sql import SparkSession
import time

# Create a SparkSession
spark = SparkSession.builder.appName("SparkSQL").getOrCreate()

In [12]:
import pandas as pd

In [33]:
pd_df = pd.read_csv("https://2u-data-curriculum-team.s3.amazonaws.com/dataviz-classroom/v1.2/22-big-data/home_sales_revised.csv")
pd_df.shape

(33287, 11)

In [34]:
pd_df.to_csv("home_sales_revised.csv", index=False)

In [35]:
# 1. Read in the AWS S3 bucket into a DataFrame.
from pyspark import SparkFiles
#url = "https://2u-data-curriculum-team.s3.amazonaws.com/dataviz-classroom/v1.2/22-big-data/home_sales_revised.csv"
df = spark.read.csv("home_sales_revised.csv", header=True, inferSchema=True)

In [36]:
df.show(n=5)

+--------------------+----------+----------+------+--------+---------+-----------+--------+------+----------+----+
|                  id|      date|date_built| price|bedrooms|bathrooms|sqft_living|sqft_lot|floors|waterfront|view|
+--------------------+----------+----------+------+--------+---------+-----------+--------+------+----------+----+
|f8a53099-ba1c-47d...|2022-04-08|      2016|936923|       4|        3|       3167|   11733|     2|         1|  76|
|7530a2d8-1ae3-451...|2021-06-13|      2013|379628|       2|        2|       2235|   14384|     1|         0|  23|
|43de979c-0bf0-4c9...|2019-04-12|      2014|417866|       2|        2|       2127|   10575|     2|         0|   0|
|b672c137-b88c-48b...|2019-10-16|      2016|239895|       2|        2|       1631|   11149|     2|         0|   0|
|e0726d4d-d595-407...|2022-01-08|      2017|424418|       3|        2|       2249|   13878|     2|         0|   4|
+--------------------+----------+----------+------+--------+---------+----------

In [37]:
df.printSchema()

root
 |-- id: string (nullable = true)
 |-- date: date (nullable = true)
 |-- date_built: integer (nullable = true)
 |-- price: integer (nullable = true)
 |-- bedrooms: integer (nullable = true)
 |-- bathrooms: integer (nullable = true)
 |-- sqft_living: integer (nullable = true)
 |-- sqft_lot: integer (nullable = true)
 |-- floors: integer (nullable = true)
 |-- waterfront: integer (nullable = true)
 |-- view: integer (nullable = true)



In [15]:
spark

In [88]:
# 2. Create a temporary view of the DataFrame.
df.createOrReplaceTempView("home_sales")


In [100]:
# 3. What is the average price for a four bedroom house sold in each year rounded to two decimal places?
from pyspark.sql.functions import mean, round
x = spark.sql("SELECT * FROM home_sales where bedrooms = 4")
x.groupBy("date_built").agg(round(mean("price"), 2)).show()

+----------+--------------------+
|date_built|round(avg(price), 2)|
+----------+--------------------+
|      2015|           307908.86|
|      2013|           299999.39|
|      2014|           299073.89|
|      2012|           298233.42|
|      2016|           296050.24|
|      2010|           296800.75|
|      2011|            302141.9|
|      2017|           296576.69|
+----------+--------------------+



In [102]:
# 4. What is the average price of a home for each year the home was built that have 3 bedrooms and 3 bathrooms rounded to two decimal places?
x = spark.sql("SELECT * FROM home_sales where bedrooms = 3 and bathrooms = 3")
x.groupBy("date_built").agg(round(mean("price"), 2)).show()

+----------+--------------------+
|date_built|round(avg(price), 2)|
+----------+--------------------+
|      2015|            288770.3|
|      2013|           295962.27|
|      2014|           290852.27|
|      2012|           293683.19|
|      2016|           290555.07|
|      2010|           292859.62|
|      2011|           291117.47|
|      2017|           292676.79|
+----------+--------------------+



In [105]:
# 5. What is the average price of a home for each year built that have 3 bedrooms, 3 bathrooms, with two floors,
# and are greater than or equal to 2,000 square feet rounded to two decimal places?
x = spark.sql("SELECT * FROM home_sales where bedrooms = 3 and bathrooms = 3 and floors = 2 and sqft_living > 2000")
x.groupBy("date_built").agg(round(mean("price"), 2)).show()


+----------+--------------------+
|date_built|round(avg(price), 2)|
+----------+--------------------+
|      2015|           297609.97|
|      2013|           303676.79|
|      2014|           297619.46|
|      2012|           307539.97|
|      2016|            293965.1|
|      2010|           285010.22|
|      2011|           276553.81|
|      2017|           280527.62|
+----------+--------------------+



In [117]:
# 6. What is the "view" rating for the average price of a home, rounded to two decimal places, where the homes are greater than
# or equal to $350,000? Although this is a small dataset, determine the run time for this query.

start_time = time.time()
avg_view = spark.sql("SELECT view FROM home_sales where price >= 350000").select(mean('view'))

print("--- %s seconds ---" % (time.time() - start_time))
avg_view.show()

--- 0.03050971031188965 seconds ---
+---------+
|avg(view)|
+---------+
|   32.264|
+---------+



In [124]:
# 7. Cache the the temporary table home_sales.
df.cache()

DataFrame[id: string, date: date, date_built: int, price: int, bedrooms: int, bathrooms: int, sqft_living: int, sqft_lot: int, floors: int, waterfront: int, view: int]

In [125]:
# 8. Check if the table is cached.
spark.catalog.isCached('home_sales')

True

In [174]:
# 9. Using the cached data, run the query that filters out the view ratings with average price
#  greater than or equal to $350,000. Determine the runtime and compare it to uncached runtime.


start_time = time.time()
avg_view = spark.sql("SELECT view FROM home_sales where price >= 350000").select((mean('view'))).select("*", round('avg(view)',2))
print("--- %s seconds ---" % (time.time() - start_time))

av = avg_view.drop("avg(view)")
av.show()


--- 0.051929473876953125 seconds ---
+-------------------+
|round(avg(view), 2)|
+-------------------+
|              32.26|
+-------------------+



In [129]:
# 10. Partition by the "date_built" field on the formatted parquet home sales data
df.write.partitionBy("date_built").mode("overwrite").parquet("home_sales.parquet")

In [130]:
# 11. Read the parquet formatted data.
parq_sales =spark.read.parquet("home_sales.parquet")
parq_sales.show(n=5)

+--------------------+----------+------+--------+---------+-----------+--------+------+----------+----+----------+
|                  id|      date| price|bedrooms|bathrooms|sqft_living|sqft_lot|floors|waterfront|view|date_built|
+--------------------+----------+------+--------+---------+-----------+--------+------+----------+----+----------+
|2ed8d509-7372-46d...|2021-08-06|258710|       3|        3|       1918|    9666|     1|         0|  25|      2015|
|941bad30-eb49-4a7...|2020-05-09|229896|       3|        3|       2197|    8641|     1|         0|   3|      2015|
|c797ca12-52cd-4b1...|2019-06-08|288650|       2|        3|       2100|   10419|     2|         0|   7|      2015|
|0cfe57f3-28c2-472...|2019-10-04|308313|       3|        3|       1960|    9453|     2|         0|   2|      2015|
|d715f295-2fbf-4e9...|2021-05-17|391574|       3|        2|       1635|    8040|     2|         0|  10|      2015|
+--------------------+----------+------+--------+---------+-----------+--------+

In [133]:
# 12. Create a temporary table for the parquet data.
spark.sql("CREATE TEMPORARY VIEW parq_sales USING parquet OPTIONS (path 'home_sales.parquet')")
spark.sql("SELECT * FROM parq_sales" ).show()

+--------------------+----------+------+--------+---------+-----------+--------+------+----------+----+----------+
|                  id|      date| price|bedrooms|bathrooms|sqft_living|sqft_lot|floors|waterfront|view|date_built|
+--------------------+----------+------+--------+---------+-----------+--------+------+----------+----+----------+
|2ed8d509-7372-46d...|2021-08-06|258710|       3|        3|       1918|    9666|     1|         0|  25|      2015|
|941bad30-eb49-4a7...|2020-05-09|229896|       3|        3|       2197|    8641|     1|         0|   3|      2015|
|c797ca12-52cd-4b1...|2019-06-08|288650|       2|        3|       2100|   10419|     2|         0|   7|      2015|
|0cfe57f3-28c2-472...|2019-10-04|308313|       3|        3|       1960|    9453|     2|         0|   2|      2015|
|d715f295-2fbf-4e9...|2021-05-17|391574|       3|        2|       1635|    8040|     2|         0|  10|      2015|
|a18515a2-86f3-46b...|2022-02-18|419543|       3|        2|       1642|   12826|

In [175]:
# 13. Run the query that filters out the view ratings with average price of greater than or equal to $350,000
# with the parquet DataFrame. Round your average to two decimal places.
# Determine the runtime and compare it to the cached version.

start_time = time.time()
avg_view = spark.sql("SELECT view FROM home_sales where price >= 350000").select((mean('view'))).select("*", round('avg(view)',2))

print("--- %s seconds ---" % (time.time() - start_time))

av = avg_view.drop("avg(view)")
av.show()

--- 0.07448339462280273 seconds ---
+-------------------+
|round(avg(view), 2)|
+-------------------+
|              32.26|
+-------------------+



In [176]:
# 14. Uncache the home_sales temporary table.
spark.catalog.clearCache()

In [177]:
# 15. Check if the home_sales is no longer cached
spark.catalog.isCached('parq_sales')


False